# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates. 
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [51]:
import cudf
import cuml

In [52]:
import numpy as np

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [53]:
gdf = cudf.read_csv('./data/week3.csv')
print(gdf.dtypes)
gdf.shape

age             int64
sex            object
employment     object
infected      float64
dtype: object


(58479894, 4)

## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [54]:
gdf['infected'] = gdf['infected'].astype('float32')

In [55]:
print(gdf.dtypes)

age             int64
sex            object
employment     object
infected      float32
dtype: object


In [98]:
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


In [57]:
gdf['employment'].unique()

0           A
1     B, D, E
2           C
3           F
4           G
5           H
6           I
7           J
8           K
9           L
10          M
11          N
12          O
13          P
14          Q
15    R, S, T
16          U
17          V
18          X
19          Z
Name: employment, dtype: object

In [58]:
employments = np.asarray(gdf['employment'].unique().to_numpy())

In [59]:
employments

array(['A', 'B, D, E', 'C', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R, S, T', 'U', 'V', 'X', 'Z'], dtype=object)

In [60]:
test = gdf[(gdf['employment'] == 'A')]

In [61]:
test.shape

(305755, 4)

In [62]:
test[test['infected']==1].shape

(1178, 4)

In [63]:
for emp in employments:
    print(emp)

A
B, D, E
C
F
G
H
I
J
K
L
M
N
O
P
Q
R, S, T
U
V
X
Z


Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [64]:
for emp in employments:
    empl[emp] = len(gdf[(gdf['employment'] == emp) & (gdf['infected'] == 1)])/len(gdf[gdf['employment'] == emp])

1178

In [65]:
empl

{'A': 0.003852757927098494,
 'B, D, E': 0.003773739946793759,
 'C': 0.003881672484213866,
 'F': 0.0031816876627218365,
 'G': 0.004947506173465579,
 'H': 0.003387583295073737,
 'I': 0.010353500473796637,
 'J': 0.003939436042196867,
 'K': 0.004771891810984617,
 'L': 0.00497012728374751,
 'M': 0.004777052804994364,
 'N': 0.0047844510096647225,
 'O': 0.005284125491064018,
 'P': 0.006190311923993122,
 'Q': 0.01275573935952277,
 'R, S, T': 0.0053900168763782825,
 'U': 0.0002168693362249245,
 'V': 0.00759006367897857,
 'X': 0.004538760797415214,
 'Z': 0.005654639190372061}

Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [66]:
{k: v for k, v in sorted(empl.items(), key=lambda item: item[1])}

{'U': 0.0002168693362249245,
 'F': 0.0031816876627218365,
 'H': 0.003387583295073737,
 'B, D, E': 0.003773739946793759,
 'A': 0.003852757927098494,
 'C': 0.003881672484213866,
 'J': 0.003939436042196867,
 'X': 0.004538760797415214,
 'K': 0.004771891810984617,
 'M': 0.004777052804994364,
 'N': 0.0047844510096647225,
 'G': 0.004947506173465579,
 'L': 0.00497012728374751,
 'O': 0.005284125491064018,
 'R, S, T': 0.0053900168763782825,
 'Z': 0.005654639190372061,
 'P': 0.006190311923993122,
 'V': 0.00759006367897857,
 'I': 0.010353500473796637,
 'Q': 0.01275573935952277}

## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [89]:
employments_m={}

In [90]:
for emp in employments:
    employments_m[emp] = len(gdf[(gdf['employment'] == emp) & (gdf['infected'] == 1) & (gdf['sex'] == 'm')]) / \
     len(gdf[(gdf['employment'] == emp) & (gdf['sex'] == 'm')])

In [91]:
employments_m

{'A': 0.002514248901693763,
 'B, D, E': 0.002461599851171619,
 'C': 0.002569246274720092,
 'F': 0.0025350454699947776,
 'G': 0.0025961112974234963,
 'H': 0.002482170908430698,
 'I': 0.005117046330160729,
 'J': 0.002546083647707825,
 'K': 0.002489851150202977,
 'L': 0.0021967686242172803,
 'M': 0.002519578789698296,
 'N': 0.0025380676082357094,
 'O': 0.0025428302565663834,
 'P': 0.002576928363542918,
 'Q': 0.005119846392814052,
 'R, S, T': 0.0025419716283166644,
 'U': 0.00010978433720486259,
 'V': 0.0036849423268991,
 'X': 0.002434716188475677,
 'Z': 0.0025427433063217823}

In [93]:
{k: v for k, v in sorted(employments_m.items(), key=lambda item: item[1])}

{'U': 0.00010978433720486259,
 'L': 0.0021967686242172803,
 'X': 0.002434716188475677,
 'B, D, E': 0.002461599851171619,
 'H': 0.002482170908430698,
 'K': 0.002489851150202977,
 'A': 0.002514248901693763,
 'M': 0.002519578789698296,
 'F': 0.0025350454699947776,
 'N': 0.0025380676082357094,
 'R, S, T': 0.0025419716283166644,
 'Z': 0.0025427433063217823,
 'O': 0.0025428302565663834,
 'J': 0.002546083647707825,
 'C': 0.002569246274720092,
 'P': 0.002576928363542918,
 'G': 0.0025961112974234963,
 'V': 0.0036849423268991,
 'I': 0.005117046330160729,
 'Q': 0.005119846392814052}

In [94]:
{k: v for k, v in sorted(empl.items(), key=lambda item: item[1])}

{'U': 0.0002168693362249245,
 'F': 0.0031816876627218365,
 'H': 0.003387583295073737,
 'B, D, E': 0.003773739946793759,
 'A': 0.003852757927098494,
 'C': 0.003881672484213866,
 'J': 0.003939436042196867,
 'X': 0.004538760797415214,
 'K': 0.004771891810984617,
 'M': 0.004777052804994364,
 'N': 0.0047844510096647225,
 'G': 0.004947506173465579,
 'L': 0.00497012728374751,
 'O': 0.005284125491064018,
 'R, S, T': 0.0053900168763782825,
 'Z': 0.005654639190372061,
 'P': 0.006190311923993122,
 'V': 0.00759006367897857,
 'I': 0.010353500473796637,
 'Q': 0.01275573935952277}

In [100]:
employments_f={}

In [101]:
for emp in employments:
    employments_f[emp] = len(gdf[(gdf['employment'] == emp) & (gdf['infected'] == 1) & (gdf['sex'] == 'f')]) / \
     len(gdf[(gdf['employment'] == emp) & (gdf['sex'] == 'f')])

In [103]:
print('Man:')
{k: v for k, v in sorted(employments_m.items(), key=lambda item: item[1])}

Man:


{'U': 0.00010978433720486259,
 'L': 0.0021967686242172803,
 'X': 0.002434716188475677,
 'B, D, E': 0.002461599851171619,
 'H': 0.002482170908430698,
 'K': 0.002489851150202977,
 'A': 0.002514248901693763,
 'M': 0.002519578789698296,
 'F': 0.0025350454699947776,
 'N': 0.0025380676082357094,
 'R, S, T': 0.0025419716283166644,
 'Z': 0.0025427433063217823,
 'O': 0.0025428302565663834,
 'J': 0.002546083647707825,
 'C': 0.002569246274720092,
 'P': 0.002576928363542918,
 'G': 0.0025961112974234963,
 'V': 0.0036849423268991,
 'I': 0.005117046330160729,
 'Q': 0.005119846392814052}

In [104]:
print('Female:')
{k: v for k, v in sorted(employments_f.items(), key=lambda item: item[1])}

Female:


{'U': 0.0003294440495835521,
 'L': 0.007221217096662867,
 'H': 0.007385432843682105,
 'N': 0.007388874673439539,
 'X': 0.007391298719790817,
 'A': 0.007491274367923725,
 'G': 0.007555738052693474,
 'F': 0.007577209017517365,
 'P': 0.007583796037232645,
 'Z': 0.007628532674673546,
 'C': 0.007630093752361246,
 'J': 0.007644520496986831,
 'M': 0.007645297567778188,
 'K': 0.007672137515781295,
 'O': 0.007718923038158275,
 'R, S, T': 0.007748406237713437,
 'B, D, E': 0.00797321551843159,
 'V': 0.010851510957458253,
 'Q': 0.014946907072570206,
 'I': 0.01506411156033882}

In [105]:
print('Without sex:')
{k: v for k, v in sorted(empl.items(), key=lambda item: item[1])}

Without sex:


{'U': 0.0002168693362249245,
 'F': 0.0031816876627218365,
 'H': 0.003387583295073737,
 'B, D, E': 0.003773739946793759,
 'A': 0.003852757927098494,
 'C': 0.003881672484213866,
 'J': 0.003939436042196867,
 'X': 0.004538760797415214,
 'K': 0.004771891810984617,
 'M': 0.004777052804994364,
 'N': 0.0047844510096647225,
 'G': 0.004947506173465579,
 'L': 0.00497012728374751,
 'O': 0.005284125491064018,
 'R, S, T': 0.0053900168763782825,
 'Z': 0.005654639190372061,
 'P': 0.006190311923993122,
 'V': 0.00759006367897857,
 'I': 0.010353500473796637,
 'Q': 0.01275573935952277}

In [ ]:
len(gd)

## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)